In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

torch.manual_seed(1228)

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

batch_size = 8

var_list = [300]

In [4]:
green_dir1 = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\green1'
green_dir2 = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\green2'
yellow_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\yellow'
red_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\red'
blue_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\blue'

green_set1 = datasets.ImageFolder(
    green_dir1,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

green_set2 = datasets.ImageFolder(
    green_dir2,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

yellow_set = datasets.ImageFolder(
    yellow_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
red_set = datasets.ImageFolder(
    red_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
blue_set = datasets.ImageFolder(
    blue_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)


train_set1, test_set1 = torch.utils.data.random_split(green_set1, [250, 50])
train_set2, test_set2 = torch.utils.data.random_split(green_set2, [250, 50])
test_set = test_set1 + test_set2

normal_set = green_set1 + green_set2
abnormal_set = red_set + yellow_set + blue_set

In [5]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))

        # Decoder        
        self.tran_cnn_layer1 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2, padding=0),
            nn.ReLU())

        self.tran_cnn_layer2 = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size = 2, stride = 2, padding=0),
            nn.Sigmoid())
            
            
    def forward(self, x):
        output = self.cnn_layer1(x)
        output = self.cnn_layer2(output)      
        output = self.tran_cnn_layer1(output)
        output = self.tran_cnn_layer2(output)
        
        return output

In [6]:
import math

normal_loader1 = torch.utils.data.DataLoader(green_set1, batch_size=batch_size, shuffle=True)
normal_loader2 = torch.utils.data.DataLoader(green_set2, batch_size=batch_size, shuffle=True)
normal_loader = torch.utils.data.DataLoader(normal_set, batch_size=batch_size, shuffle=True)
abnormal_loader = torch.utils.data.DataLoader(abnormal_set, batch_size=batch_size, shuffle=True)

criterion = nn.MSELoss()

def myEquation4(x, model, normal_loader = normal_loader):    
    normal_err, abnormal_err = 0, 0
    
    for i, (X_normal, _) in enumerate(normal_loader):
        X_normal = X_normal.cuda()
        output = model(X_normal)
        loss = criterion(X_normal, output)

        if loss.item() > x:
            normal_err += 1
        
    for i, (X_abnormal, _) in enumerate(abnormal_loader):
        X_abnormal = X_abnormal.cuda()
        output = model(X_abnormal)
        loss = criterion(X_abnormal, output)

        if loss.item() < x:
            abnormal_err += 1

    #err = ((normal_err/len(normal_loader)) + (abnormal_err/len(abnormal_loader)))/2.0
    err = (normal_err + abnormal_err) / (len(normal_loader) + len(abnormal_loader))
    
    return err


gr = (math.sqrt(5) + 1) / 2

def gss(f, a, b, model, loader=normal_loader, tol=1e-12):
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c, model, normal_loader=loader) < f(d, model, normal_loader=loader):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr

    return (b + a) / 2

In [7]:
model = ConvAutoEncoder().to(device)
model = torch.load(f'model_per_round/round_300_weight.pth')

threshold = gss(myEquation4, 0.0005295865061149622, 0.0015098906197171245, model, normal_loader)
err = myEquation4(threshold, model)
print(f"FedAvg\nthreshold : {threshold:.5f}\nerr : {err:.5f}\n")
with open(f'data/threshold by gss.txt', 'a') as f:
    f.write(f'{threshold}\n')


minimum = [0.0013358147298761953, 0.000579554884073635]
maximum = [0.003870297086218165, 0.0022228552064754896]

for i, loader in enumerate([normal_loader1, normal_loader2]):
    model = ConvAutoEncoder().to(device)
    model = torch.load(f'each_model/model{i+1}.pth')

    threshold = gss(myEquation4, minimum[i], maximum[i], model, loader=loader)
    err = myEquation4(threshold, model)
    print(f"model{i+1}\nthreshold: {threshold:.5f}\nerr: {err:.5f}\n")

    with open(f'data/threshold by gss.txt', 'a') as f:
        f.write(f'{threshold}\n')

FedAvg
threshold : 0.00076
err : 0.01667

model1
threshold: 0.00235
err: 0.00333

model2
threshold: 0.00113
err: 0.00000



In [6]:
minimum = [0.0005373578149010427, 0.00010975637537500773, 0.0009650923020672053]
maximum = [0.001579382724335624, 0.0015793827114005884, 0.0015793827192909602]

for i, loader in enumerate([normal_loader, normal_loader1, normal_loader2]):
    model = ConvAutoEncoder().to(device)
    model = torch.load(f'model_per_round/round_300_weight.pth')

    threshold = gss(myEquation4, minimum[i], maximum[i], model, loader)
    err = myEquation4(threshold, model)

    if i==0:
        print(f"FedAvg\nthreshold : {threshold:.5f}\nerr : {err:.5f}\n")
    else:
        print(f"Client{i}\nthreshold : {threshold:.5f}\nerr : {err:.5f}\n")

    with open(f'data/threshold by gss.txt', 'a') as f:
        f.write(f'{threshold}\n')


minimum = [0.0010596374647381405, 0.000543640300941964]
maximum = [0.0025076100752792424, 0.0013678904930647048]

for i, loader in enumerate([normal_loader1, normal_loader2]):
    model = ConvAutoEncoder().to(device)
    model = torch.load(f'each_model/model{i+1}.pth')

    threshold = gss(myEquation4, minimum[i], maximum[i], model, loader=loader)
    err = myEquation4(threshold, model)
    print(f"model{i+1}\nthreshold: {threshold:.5f}\nerr: {err:.5f}\n")

    with open(f'data/threshold by gss.txt', 'a') as f:
        f.write(f'{threshold}\n')

FedAvg
threshold : 0.00076
err : 0.01667

Client1
threshold : 0.00058
err : 0.08333

Client2
threshold : 0.00098
err : 0.07333

model1
threshold: 0.00211
err: 0.02667

model2
threshold: 0.00117
err: 0.00000



In [ ]:
model = ConvAutoEncoder().to(device)
model = torch.load(f'model_per_round/round_300_weight.pth')

#threshold = 0.00113
threshold = 0.00055
err = myEquation4(threshold, model)

print(f"Num Rounds: 300\nthreshold : {threshold:.5f}\nerr : {err:.5f}\n")

Num Rounds: 100
threshold : 0.00113
err : 0.00000

